In [ ]:
import numpy as np

# Constants
D2R = np.pi / 180  # Degrees to radians conversion

class GNSS:
    def __init__(self):
        self.time = 0.0  # Time
        self.blh = np.zeros(3)  # Latitude, longitude, height
        self.std = np.zeros(3)  # Standard deviation of position

class GnssFileLoader:
    def __init__(self, filename, columns=7):
        self.file_loader = FileLoader(filename, columns, 'TEXT')
        self.columns = columns
        self.gnss = GNSS()
    
    def next(self):
        data = self.file_loader.load()
        if not data:
            return None

        self.gnss.time = data[0]
        self.gnss.blh[:] = data[1:4]
        
        if len(data) == 7:
            self.gnss.std[:] = data[4:7]
        else:
            self.gnss.std[:] = data[7:10]

        self.gnss.blh[0] *= D2R  # Convert latitude from degrees to radians
        self.gnss.blh[1] *= D2R  # Convert longitude from degrees to radians

        return self.gnss

# You need to have a FileLoader class defined as in previous examples
# Here's a basic structure for FileLoader again for reference
class FileLoader:
    def __init__(self, filename, columns, filetype):
        self.filename = filename
        self.columns = columns
        self.filetype = filetype
        self.file = None
        self.open(filename, columns, filetype)
    
    def open(self, filename, columns, filetype):
        mode = 'r' if filetype == 'TEXT' else 'rb'
        try:
            self.file = open(filename, mode)
            return True
        except IOError:
            print("File could not be opened")
            return False
    
    def load(self):
        if self.is_eof():
            return None

        line = self.file.readline()
        if not line:
            return None
        
        # Assuming data is separated by commas
        splits = [float(x.strip()) for x in line.split(',')]
        return splits
    
    def is_eof(self):
        # Checks if we are at the end of the file
        current_position = self.file.tell()
        self.file.seek(0, os.SEEK_END)
        end = self.file.tell()
        self.file.seek(current_position)
        return current_position == end

# Usage
gnss_loader = GnssFileLoader('gnss_data.txt', 7)
gnss_data = gnss_loader.next()
print(gnss_data.blh, gnss_data.std, gnss_data.time)
